In [1]:
!pip install transformers trl accelerate torch bitsandbytes peft sentencepiece wandb datasets -qU 
!pip install huggingface-hub -qU

In [7]:
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [11]:
tuned_model = AutoModelForCausalLM.from_pretrained(
    "gouthamsk/mistral-embedded-c-v0.3",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "gouthamsk/mistral-embedded-c-instruct-v0.4",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained("gouthamsk/mistral-embedded-c-instruct-v0.4")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [3]:
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained("gouthamsk/mistral_embedded_c")
tuned_model = PeftModel.from_pretrained(model, "gouthamsk/mistral_embedded_c")

adapter_config.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

In [3]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [12]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Tue Apr  9 08:44:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0              27W /  70W |  13883MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
tuned_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024,

In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd
df = pd.read_csv(r'/Users/biboxdev/Desktop/research/codeBert/fine_tuning/embededd_new.csv')
dataset = Dataset.from_pandas(df)

dataset.push_to_hub("gouthamsk/embedded_dataset_mixed_small")

In [4]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)
  return decoded_output[0]

In [9]:
generate_response("write a program to blink a led for ESP32",model)

'<s> write a program to blink a led for ESP32 with  1 second delay\n\n#include <stdio.h>\n#include "freertos/FreeRTOS.h"\n#include "freertos/task.h"\n#include <driver/gpio.h>\n#define LED_PIN GPIO_NUM_2 // Define the GPIO pin to which the LED is connected\nvoid app_main() {\n    // Configure the GPIO pin as output\n    esp_err_t result = gpio_config(LED_PIN, &gpio_config_t{\n        .pin_bit_mask = (1ULL << LED_PIN),\n        .mode = GPIO_MODE_OUTPUT,\n    });\n    if (result != ESP_OK) {\n        printf("GPIO configuration failed with error: %s\\n",\n                esp_err_to_name(result));\n        exit(1);\n    }\n    while (1) {\n        // Turn the LED on\n        gpio_set_level(LED_PIN, 1);\n        // Wait for 1 second\n        vTaskDelay(1000 / portTICK_PERIOD_MS);\n        // Turn the LED off\n        gpio_set_level(LED_PIN, 0);\n        // Wait for 1 second\n        vTaskDelay(1000 / portTICK_PERIOD_MS);\n    }\n}</s>'

In [10]:
generate_response("write a program to blink a led for ESP32",base_model)

'<s> write a program to blink a led for ESP32 using the Arduino IDE\n\nTo blink an LED on an ESP32 using the Arduino IDE, follow these steps:\n\n1. Set up the ESP32 development environment:\n   Install the Arduino IDE, then add the BoardManager.json and ESP32 board files from the official ESP32 repository: https://github.com/espressif/arduino-esp32\n\n2. Create a new Arduino sketch:\n   In Arduino IDE, create a new sketch by clicking "File" -> "New" or by using the keyboard shortcut "Ctrl+N".\n\n3. Connect the LED and resistor:\n   Connect the LED to an digital I/O pin on the ESP32 using a 220Ω resistor.\n\n4. Write the Arduino code:\n   In your new sketch, paste the following code:\n\n```C++\n#define LED_PIN 2\n\nvoid setup() {\n  pinMode(LED_PIN, OUTPUT); // configure LED_PIN as an output\n}\n\nvoid loop() {\n  digitalWrite(LED_PIN, HIGH); // turn the LED on (HIGH is the voltage level)\n  delay(1000); // wait for a second\n  digitalWrite(LED_PIN, LOW);  // turn the LED off by making 

In [17]:
generate_response("how many interrupt do ESP32 have",base_model)

'<s> ESP HTTP Client Overview esp_http_client component provides a set of APIs to easily send, post and upload data to various HTTP and WebSocket servers. It uses the FreeRTOS+ TCP transport layer and is designed for both HTTP and WebSocket connections. This component is lightweight and minimalistic, using only required features with efficient memory usage. The APIs are designed for both synchronous and asynchronous use cases, which allows the user to choose the best solution for a particular use case. Key features: * Uses FreeRTOS+TCP transport layer for reliable connection and event handling * Supports both HTTP and HTTPS protocols * Supports both binary and text response handling * Supports asynchronous and synchronous requests * Supports multiple ongoing transfers * Supports HTTP and HTTPS Keep-Alive * Supports automatic redirection following 3xx status codes * Supports streaming data over HTTP, using the chunked transfer encoding * Supports reading and writing Cookies* Supports ba

In [13]:
generate_response("Delete a peer from peer list. - Parameters peer_addr -- peer MAC address ",model)

'<s> Delete a peer from peer list. - Parameters peer_addr -- peer MAC address 0x00:00:00:00:00:00 peer_index -- the peer index in peer list 0,1,2 ... peer_num -1: delete a peer from peer list successfully, else fail.  Return values: -ESP_OK succeed -1: delete a peer from peer list failed because no such peer in list ESP_ERR_INVALID_ARG : invalid argument ESP_ERR_NOT_FOUND : not found the peer in list ESP_ERR_ invalid : invalid argument ESP_ERR_NO_MEM : no memory to allocate ESP_ERR_TIMEOUT : timeout to delete a peer from peer list\n-\n-\nesp_ble_mesh_add_static_mesh_addr(peer_addr, static_mesh_addr, static_mesh_addr_length, callback, user_ctx)\nAdd permanent mesh address to a peer.\n</s>'

In [6]:
generate_response("The Light LC (Lightness Control) Server model extends the Light Lightness Server model and the Generic",model)

'<s><s> The Light LC (Lightness Control) Server model extends the Light Lightness Server model and the Generic Server model. In this model, a server device with a light sensor can control the light level of a target device over Bluetooth Low Energy (BLE).\n\n# Architecture Overview\nThe following figure shows an architecture of the Light LC Server model.\n The application code contains the following components:\n- Bluetooth Lightness Service (BLS) – defines service/characteristic UUIDs, and provides access to the service/characteristic data by an application framework.\n- Attributes and Descriptors – the Light LC model uses various characteristics and descriptors.\n- Application Framework – provides a user interface to interact with the BLS instance.\n  Other architecture components that are not part of an application code are as follows:\n- Bluetooth LE API (bluetooth/bluetooth-le) – contains an API for BLE specific features.\n- Platform Abstraction Layer (PAL) (bluetooth/bluetooth-le

In [11]:
generate_response("ESP-MQTT Overview ESP-MQTT is an implementation of MQTT protocol client, which is a",base_model)

'<s> ESP-MQTT Overview ESP-MQTT is an implementation of MQTT protocol client, which is a lightweight, publish/subscribe messaging protocol that allows you to exchange data between devices using asynchronous methods. In this tutorial, you will learn how to create a simple web application to control an Espressif ESP8266 microcontroller using MQTT publish/subscribe messages. Prerequisites Before following this tutorial, make sure that you have the following:\n\n* A computer with a stable internet connection\n* Latest Arduino IDE with ESP8266 board support package installed\n* Basic understanding of MQTT protocol, C programming, and HTML/JavaScript\n\nHardware Requirements\n\n* ESP8266 board (e.g., NodeMCU, Wemos D1 Mini)\n* A breadboard, wires, and some LEDs (optional)\n\nStep 1: Set Up the ESP8266 Board\nCreate a new Arduino project and select the appropriate board. In this example, we will use the following board configuration:\n\n* Board: NodeMCU 1.0 (ESP-12)\n* processor: ESP8266 with

In [12]:
generate_response("Delete a peer from peer list. - Parameters peer_addr -- peer MAC address ",base_model)

'<s> Delete a peer from peer list. - Parameters peer_addr -- peer MAC address --------------------------- - code block-------------------------- ```lua function delete_peer(peer_addr) -- check if peer is in the peer list if not, error return _,_,_, peer in peer_list then -- remove peer from the table peer_list[peer_addr] = nil end end ``` Note: This code snippet assumes that `peer_list` is a table that maps peer MAC addresses to their network interfaces. You may need to modify the function `delete_peer` according to your implementation.</s>'

In [2]:
from datasets import load_dataset, Dataset
import pandas as pd
import pandas as pd
df_mixed = pd.read_csv(r'/Users/biboxdev/Desktop/research/codeBert/fine_tuning/dataset_new - train.csv')
df = pd.read_csv(r'/Users/biboxdev/Desktop/research/codeBert/fine_tuning/mistal_training/c_dataset.csv')
df.drop(columns=['lang','resource'], inplace=True)



/Users/biboxdev/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
df = df[:150]
df

,query,answer
0,Write a C program to print the elements of a t...,Here is a C program that prints the elements o...
1,Write a C program that prompts the user to ent...,Here is an example C program that prompts the ...
2,Please generate a system variable in C languag...,Implementing a complete date and time system t...
3,Write a C program to calculate the order of a ...,Here's a C program that meets the requirements...
4,Build a program in C to find all prime numbers...,Here is an implementation of the program in C:...
...,...,...
145,Write a function in C that swaps the first and...,Here is the code for the function in C that sw...
146,Convert the given linear equation ax + b = c i...,It is not possible to convert the given linear...
147,Re-write the code in the given input to calcul...,def fibonacci(n):\n if n <= 1:\n ret...
148,Create a function in C that takes a string as ...,Here is a possible implementation of the funct...


In [15]:

df.rename(columns={'query': 'instruction'}, inplace=True)
df.rename(columns={'answer': 'output'}, inplace=True)
result = pd.concat([df, df_mixed])
# df.head()
result = result.reset_index(drop=True)
result
dataset = Dataset.from_pandas(result)
dataset_shuffled = dataset.shuffle(seed=42)
dataset_shuffled
dataset_shuffled.push_to_hub("gouthamsk/embedded_dataset_mixed_small")



Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.46s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/gouthamsk/embedded_dataset_mixed_small/commit/53a4f199d00546ef2a0a22809aaab24cbd76145e', commit_message='Upload dataset', commit_description='', oid='53a4f199d00546ef2a0a22809aaab24cbd76145e', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
!pip3 install matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Using cached pyparsing-3.1.1-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 10.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 12.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.0 MB/s eta 0:00:00a 0:00:01
Using cached pyparsing-3.1.1-py3-none-any.whl (103 kB)


In [19]:
from datasets import load_dataset, Dataset

def create_text_row(data):
    if(input==None):
        text_row = f"""<s>[INST]{data['instruction']}[/INST]\\n{data['output']}</s>"""
    else :
        text_row = f"""<s>[INST]{data['instruction']} with {data['input']} [/INST] \n {data['output']}</s>"""
    return text_row

def prepare_train_data(data_id):
    data = load_dataset(data_id, split="train")
    data_df = data.to_pandas() 
    data_df["text"] =data_df.apply(create_text_row, axis =1) 
    data = Dataset.from_pandas(data_df)
    return data 

In [20]:

instruct_tune_dataset = prepare_train_data("gouthamsk/embedded_dataset_mixed_small")

Generating train split: 100%|██████████| 452/452 [00:00<00:00, 11122.08 examples/s]


In [22]:
import pandas as pd
import matplotlib.pyplot as plt

# Sample DataFrame
# data = {'text': ['Hello world', 'This is a test']}
df = pd.DataFrame(instruct_tune_dataset)

# Calculate token length
df['token_length'] = df['text'].apply(lambda x: len(x.split()))


# Plotting
plt.hist(df['token_length'], bins=range(1, max(df['token_length']) + 2), edgecolor='black')
plt.xlabel('Token Length')
plt.ylabel('Frequency')
plt.title('Token Length Distribution')
plt.xticks(range(1, max(df['token_length']) + 1))
plt.grid(True)
plt.show()


KeyError: 'text'

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_embedded_c_v0.2",
  #num_train_epochs=10,
  max_steps = 300, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 4,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  # evaluation_strategy="steps",
  # eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  bf16=True,
  lr_scheduler_type='constant',
  report_to="wandb",
)